# Interactive Wave Simulation

This notebook demonstrates the wave dynamics simulation and explores connections to machine learning concepts.

## Setup

First, let's import the necessary modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from string_model import String
from solver import CentralDifferenceSolver, RK4Solver, VerletSolver
import visualization
import analysis

# Make plots appear in notebook
%matplotlib inline

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 7)

## Example 1: Basic Wave Propagation

Let's simulate a simple Gaussian pulse propagating on a uniform string.

### Physics Concept

We're solving the wave equation:

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$$

where $c = \sqrt{T/\mu}$ is the wave speed.

### ML Connection

Wave propagation is analogous to **forward propagation in neural networks** - information flows through layers just like waves propagate through space.

In [ ]:
# Create uniform string
string = String(
    length=50.0,
    num_points=500,
    tension=50.0,
    density_profile='uniform',
    density_uniform=0.01
)

# Set initial Gaussian pulse
string.set_initial_gaussian(center=25.0, width=5.0, amplitude=1.0)

# Create solver
solver = CentralDifferenceSolver(string, enable_force=False)

# Solve
print("Simulating wave propagation...")
displacement_history = solver.solve(total_time=3.0, dt=0.001, save_interval=50, verbose=False)

print(f"Simulation complete! Generated {len(displacement_history)} time snapshots.")

In [ ]:
# Visualize wave evolution
visualization.plot_wave_evolution(
    displacement_history,
    string.x,
    time_history=solver.time_history,
    num_snapshots=8
)
plt.show()

## Example 2: Energy Conservation

### Physics Concept

Total energy should be conserved:
$$E_{total} = E_{kinetic} + E_{potential} = \text{constant}$$

### ML Connection

Energy conservation in physics ↔ **Loss stability in ML training**

Just as physicists monitor energy drift to detect numerical errors, we should monitor loss stability to detect training issues.

In [ ]:
# Analyze energy conservation
energy_stats = analysis.analyze_energy_conservation(
    solver.time_history,
    solver.energy_history,
    plot=True
)
plt.show()

print(f"\nEnergy Conservation Analysis:")
print(f"  Initial Energy: {energy_stats['initial_energy']:.6e}")
print(f"  Final Energy:   {energy_stats['final_energy']:.6e}")
print(f"  Relative Drift: {energy_stats['relative_drift']*100:.3f}%")
print(f"  Status:         {energy_stats['status']}")

## Example 3: Whip Dynamics (Wave Acceleration)

### Physics Concept

The famous "whip crack" occurs when density tapers:
$$c(x) = \sqrt{\frac{T}{\mu(x)}} \to \infty \text{ as } \mu(x) \to 0$$

The wave accelerates dramatically as it approaches the tip!

### ML Connection

This is analogous to **exploding gradients** in deep networks:
- High density (thick whip base) → slow wave → vanishing gradient
- Low density (thin whip tip) → fast wave → exploding gradient

**Solution:** Skip connections (ResNets) act as "wave guides" for stable propagation!

In [ ]:
# Create tapered string (whip)
whip = String(
    length=100.0,
    num_points=1000,
    tension=100.0,
    density_profile='tapered',
    density_base=0.02,
    density_tip=0.002,
    taper_exponent=1.5
)

# Set initial pulse at thick end
whip.set_initial_pulse(position=10.0, amplitude=2.0, width=3.0)

# Visualize density profile
visualization.plot_density_profile(
    whip.x,
    whip.density,
    whip.wave_speed
)
plt.show()

In [ ]:
# Simulate whip dynamics
whip_solver = VerletSolver(whip, enable_force=False)

print("Simulating whip crack...")
whip_displacement = whip_solver.solve(
    total_time=1.0,
    dt=0.0004,
    save_interval=25,
    verbose=False
)

print(f"Simulation complete!")

In [ ]:
# Visualize whip evolution
visualization.plot_wave_evolution(
    whip_displacement,
    whip.x,
    time_history=whip_solver.time_history,
    num_snapshots=8
)
plt.title("Whip Wave Propagation (Notice acceleration toward tip!)")
plt.show()

## Example 4: Comparing Numerical Integrators

### Physics Concept

Different numerical integrators have different properties:
- **Central Difference**: Simple, 2nd order accurate
- **RK4**: More accurate (4th order), more expensive
- **Velocity Verlet**: Symplectic, conserves energy better

### ML Connection

This is analogous to **different optimizers in ML**:
- **SGD**: Simple, like Euler method
- **SGD + Momentum**: Like Velocity Verlet (uses velocity)
- **Adam**: Like adaptive timestep methods

In [ ]:
# Create test string
test_string = String(
    length=50.0,
    num_points=500,
    tension=50.0,
    density_profile='uniform'
)
test_string.set_initial_gaussian(center=25.0, width=5.0, amplitude=1.0)

solvers = {
    'Central Difference': CentralDifferenceSolver,
    'RK4': RK4Solver,
    'Verlet': VerletSolver
}

results = {}

for name, SolverClass in solvers.items():
    print(f"Running {name}...")
    
    # Fresh string for each solver
    s = String(length=50.0, num_points=500, tension=50.0, density_profile='uniform')
    s.set_initial_gaussian(center=25.0, width=5.0, amplitude=1.0)
    
    solver_inst = SolverClass(s, enable_force=False)
    disp = solver_inst.solve(total_time=2.0, dt=0.001, save_interval=200, verbose=False)
    
    results[name] = disp
    
    # Energy drift
    energy_stats = analysis.analyze_energy_conservation(
        solver_inst.time_history,
        solver_inst.energy_history,
        plot=False
    )
    print(f"  Energy drift: {energy_stats['relative_drift']*100:.3f}%")

print("\nNote: Verlet has best energy conservation (symplectic integrator)!")

In [ ]:
# Compare final states
visualization.create_comparison_plot(
    results,
    test_string.x,
    time_index=-1
)
plt.show()

## Example 5: Space-Time Heatmap (Information Flow Visualization)

### Physics Concept

A space-time diagram shows how waves propagate and interfere.

### ML Connection

This is analogous to **activation heatmaps in neural networks** - showing how information flows through layers and time.

In [ ]:
# Create heatmap of wave propagation
analysis.plot_spacetime_heatmap(
    solver.displacement_history,
    string.x,
    solver.time_history
)
plt.show()

print("\nNotice:")
print("  - Diagonal lines = wave fronts propagating")
print("  - Reflections at boundaries (t ≈ 1.5s)")
print("  - Interference patterns when waves meet")
print("\nIn ML: Attention heatmaps show similar information flow patterns!")

## Key Takeaways: Physics → ML Connections

| Physics Concept | ML Analog | Why It Matters |
|-----------------|-----------|----------------|
| **Wave propagation** | Forward/backward pass | Information flow through networks |
| **Energy conservation** | Loss stability | Detect training issues early |
| **Wave speed variation** | Gradient scaling | Understand vanishing/exploding gradients |
| **Energy density** | Attention weights | Where information concentrates |
| **Numerical integrators** | Optimizers | Trade-offs in stability vs accuracy |
| **CFL condition** | Adaptive learning rates | Prevent instability |
| **Observable quantities** | Interpretability | Make systems understandable |

---

## Learn More

- **ML_CONNECTIONS.md**: Detailed technical connections with code examples
- **README.md**: Project overview and mathematical foundations
- **main.py**: Full simulation examples

---

*"The best ML architectures respect the physics of information flow."*